In [41]:
#seeding for comparing experiment in part 2
import torch
import json
SEED = 2003
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda:7')

# SNLI Bert
## Second Tutorial
https://towardsdatascience.com/fine-tuning-pre-trained-transformer-models-for-sentence-entailment-d87caf9ec9db
Check his Github code for complete notebook. I never referred to it. Medium was enough.
BERT in keras-tf: https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b

In [42]:
# define macros
BERT_MODEL = "bert-base-cased"

batch_size = 4
batches_per_epoch = 1700

# Prepare data

## load the dataset

In [43]:
# !wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip

In [44]:
import pandas as pd

# custom reader needed to handle quotechars
def read_df_custom(file):
    header = 'doc     unit1_toks      unit2_toks      unit1_txt       unit2_txt       s1_toks s2_toks unit1_sent      unit2_sent      dir     nuc_children    sat_children    genre   u1_discontinuous        u2_discontinuous       u1_issent        u2_issent       u1_length       u2_length       length_ratio    u1_speaker      u2_speaker      same_speaker    u1_func u1_pos  u1_depdir       u2_func u2_pos  u2_depdir       doclen  u1_position      u2_position     percent_distance        distance        lex_overlap_words       lex_overlap_length      unit1_case      unit2_case      label'
    extracted_columns = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label', 'distance', 'u1_depdir', 'u2_depdir', 'u2_func', 'u1_position', 'u2_position', 'sat_children', 'nuc_children']
    header = header.split()
    df = pd.DataFrame(columns=extracted_columns)
    file = open(file, 'r')

    rows = []
    count = 0 
    for line in file:
        line = line[:-1].split('\t')
        count+=1
        if count ==1: continue
        row = {}
        for column in extracted_columns:
            index = header.index(column)
            row[column] = line[index]
        rows.append(row)

    df = pd.concat([df, pd.DataFrame.from_records(rows)])
    return df

# we only need specific columns
# train_df = read_df_custom('../../processed/nld.rst.nldt_train_enriched.rels')
# test_df = read_df_custom('../../processed/nld.rst.nldt_test_enriched.rels')
# val_df = read_df_custom('../../processed/nld.rst.nldt_dev_enriched.rels')
train_df = read_df_custom('../../processed/eng.rst.gum_train_enriched.rels')
test_df = read_df_custom('../../processed/eng.rst.gum_test_enriched.rels')
val_df = read_df_custom('../../processed/eng.rst.gum_dev_enriched.rels')
# train_df = read_df_custom('../processed/deu.rst.pcc_train_enriched.rels')
# test_df = read_df_custom('../processed/deu.rst.pcc_test_enriched.rels')
# val_df = read_df_custom('../processed/deu.rst.pcc_dev_enriched.rels')

## Clean the data

In [45]:
#dropping any empty values
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)
test_df.dropna(inplace=True)

## Prepare a dataset handler class

In [46]:
train_df.head()

,unit1_txt,unit1_sent,unit2_txt,unit2_sent,dir,label,distance,u1_depdir,u2_depdir,u2_func,u1_position,u2_position,sat_children,nuc_children
0,Aesthetic Appreciation and Spanish Art :,Aesthetic Appreciation and Spanish Art :,In this study we used eye-tracking in the firs...,In this study we used eye-tracking in the firs...,1>2,preparation,52,ROOT,ROOT,root,0.0,0.798,1,10
1,Aesthetic Appreciation and Spanish Art :,Aesthetic Appreciation and Spanish Art :,Insights from Eye-Tracking,Insights from Eye-Tracking,1<2,elaboration,1,ROOT,ROOT,root,0.0,0.01,0,1
2,Claire Bailey-Ross claire.bailey-ross@port.ac....,Claire Bailey-Ross claire.bailey-ross@port.ac....,In this study we used eye-tracking in the firs...,In this study we used eye-tracking in the firs...,1>2,attribution,50,ROOT,ROOT,root,0.014,0.798,3,10
3,Claire Bailey-Ross claire.bailey-ross@port.ac....,Claire Bailey-Ross claire.bailey-ross@port.ac....,Andrew Beresford a.m.beresford@durham.ac.uk Du...,Andrew Beresford a.m.beresford@durham.ac.uk Du...,1<2,joint,1,ROOT,ROOT,root,0.014,0.026,0,3
4,Claire Bailey-Ross claire.bailey-ross@port.ac....,Claire Bailey-Ross claire.bailey-ross@port.ac....,Daniel Smith daniel.smith2@durham.ac.uk Durham...,Daniel Smith daniel.smith2@durham.ac.uk Durham...,1<2,joint,2,ROOT,ROOT,root,0.014,0.037,0,3


In [47]:
train_df['sat_children'].unique()

array(['1', '0', '3', '2', '6', '4', '5', '8', '7', '10', '9', '15', '11',
       '14', '18', '13', '12', '20', '17', '16'], dtype=object)

In [48]:

from multiprocessing.sharedctypes import Value
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from sys import path
path.append('/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/allennlp/data/data_loaders/')
from allennlp.data import allennlp_collate#, DataLoader
# from allennlp.data.data_loaders.simple_data_loader import SimpleDataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer
import pandas as pd

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df):
    self.num_labels = -1
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    self.tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=True) # Using a pre-trained BERT tokenizer to encode sentences
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.train_idx = None
    self.val_idx = None
    self.test_idx = None
    self.init_data()

  def init_data(self):
    self.get_label_mapping()
    self.train_data, self.train_idx = self.load_data(self.train_df)
    self.val_data, self.val_idx = self.load_data(self.val_df)
    self.test_data, self.test_idx = self.load_data(self.test_df)

  def get_label_mapping(self):
    labels = {}
    labels_list = list(set(list(self.train_df['label'].unique()) + list(self.test_df['label'].unique()) + list(self.val_df['label'].unique())))
    for i in range(len(labels_list)):
        labels[labels_list[i]] = i
    self.label_dict = labels# {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    # needed later for classification report object to generate precision and recall on test dataset
    self.rev_label_dict = {self.label_dict[k]:k for k in self.label_dict.keys()} 
  
  def add_directionality(self, premise, hypothesis, dir):
    if dir == "1<2":
        hypothesis = 'left ' + hypothesis + ' {'
        # hypothesis = '< ' + hypothesis + ' {'
        # hypothesis = '{ ' + hypothesis + ' {'
    else:
        premise = '} ' + premise + ' right'
        # premise = '} ' + premise + ' >'
        # premise = '} ' + premise + ' }'
    return premise, hypothesis

  def get_distance(self, d):
    if d<-8: return -2.0
    elif d>=-8 and d<-2: return -1.0
    elif d>=-2 and d<0: return 0.0
    elif d>=0 and d<2: return 1.0
    elif d>=2 and d<8: return 2.0
    elif d>=8: return 3.0

  def get_dep(self, d):
    if d=='ROOT': return 0.0
    elif d=='RIGHT': return 1.0
    elif d=='LEFT': return -1.0
    else: raise ValueError()

  def get_u2_func(self, u):
    u2_dict = {'root':0.0, 'conj':1.0, 'advcl':2.0, 'acl':3.0, 'xcomp':4.0, 'obl':5.0, 'ccomp':6.0,
       'parataxis':7.0, 'advmod':8.0, 'dep':9.0, 'csubj':10.0, 'nmod':11.0, 'punct':12.0, 'cc':13.0,
       'appos':14.0, 'aux':15.0, 'obj':16.0, 'iobj':17.0, 'nsubj':18.0, 'nsubj:pass':19.0, 'csubj:pass':20.0, 
       'amod':21.0, 'flat':22.0, 'acl:relcl':23.0, 'det':24.0, 'orphan':25.0, 'nummod':26.0,
       'obl:agent':27.0, 'fixed':28.0, 'mark':29.0, 'cop':30.0, 'nmod:poss':31.0, 'case':32.0,
       'compound:lvc':33.0, 'nmod:tmod':34.0, 'obl:tmod':35.0, 'reparandum':36.0, 'discourse':37.0,
       'dislocated':38.0, 'obl:npmod':39.0}
    try: result = u2_dict[u]
    except: result = 40.0
    return result

  def get_u_position(self, u):
    if u>=0.0 and u<0.1: return -5.0
    elif u>=0.1 and u<0.2: return -4.0
    elif u>=0.2 and u<0.3: return -3.0
    elif u<=0.3 and u<0.4: return -2.0
    elif u<=0.4 and u<0.5: return -1.0
    elif u<=0.5 and u<0.6: return 0.0
    elif u<=0.6 and u<0.7: return 1.0
    elif u<=0.7 and u<0.8: return 2.0
    elif u<=0.8 and u<0.9: return 3.0
    elif u<=0.9 and u<1.0: return 4.0
    elif u<=1.0 and u<1e9: return 5.0

  def get_feature(self, features):
    distance = self.get_distance(float(features[0]))
    u1_depdir = self.get_dep(features[1])
    u2_depdir = self.get_dep(features[2])
    u2_func = self.get_u2_func(features[3])
    u1_position = self.get_u_position(float(features[4]))
    u2_position = self.get_u_position(float(features[5]))
    sat_children = float(features[6])
    nuc_children = float(features[6])
    return [distance, u1_depdir, u2_depdir, u2_func, u1_position, u2_position, sat_children, nuc_children]

  def load_data2(self, df):
    MAX_LEN = 256 # dont need to enforce this now because snli is a sanitized dataset where sentence lenghts are reasonable. otherwise the beert model doesn't have enough parameters to handle long length sentences
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []
    idx = []
    idx_map = {}
    # self.reach = 'reach'

    premise_list = df['unit1_txt'].to_list()
    hypothesis_list = df['unit2_txt'].to_list()
    label_list = df['label'].to_list()
    dir_list = df['dir'].to_list()
    
    self.num_labels = max(self.num_labels, len(df['label'].unique()))

    count=0
    for (premise, hypothesis, label, dir) in zip(premise_list, hypothesis_list, label_list, dir_list):
      print('old: ', premise, hypothesis)
      premise, hypothesis = self.add_directionality(premise, hypothesis, dir)
      print('new:', premise, hypothesis, '\n')
    

  def load_data(self, df):
    MAX_LEN = 256 # dont need to enforce this now because snli is a sanitized dataset where sentence lenghts are reasonable. otherwise the beert model doesn't have enough parameters to handle long length sentences
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []
    feats = []
    idx = []
    idx_map = {}

    #"u1_depdir", "u2_depdir", "u2_func", "u1_position", "u2_position","sat_children", "nuc_children"
    
    self.num_labels = max(self.num_labels, len(df['label'].unique()))

    count=0
    # for (premise, hypothesis, label, dir, distance, u1_depdir) in zip(premise_list, hypothesis_list, label_list, dir_list, u1_depdir_list, feat_list):
    for row in df.iterrows():
      row = row[1]
      premise = row['unit1_txt']
      hypothesis = row['unit2_txt']
      label = row['label']
      dir = row['dir']

      distance = row['distance']
      u1_depdir = row['u1_depdir']
      u2_depdir = row['u2_depdir']
      u2_func = row['u2_func']
      u1_position = row['u1_position']
      u2_position = row['u2_position']
      sat_children = row['sat_children']
      nuc_children = row['nuc_children']
      features = [distance, u1_depdir, u2_depdir, u2_func, u1_position, u2_position, sat_children, nuc_children]

      premise, hypothesis = self.add_directionality(premise, hypothesis, dir)
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False, max_length=MAX_LEN, truncation=True)
      print(premise)
      print(premise_id)
      # print(self.tokenizer.encode("< "))
      # print(self.tokenizer.encode("< this"))
      # print(self.tokenizer.encode("this ."))
      # print(self.tokenizer.encode("this } "))
      # print(self.tokenizer.encode("fffffff } "))
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False, max_length=MAX_LEN, truncation=True)
      # pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      pair_token_ids = self.tokenizer.encode(premise, hypothesis, add_special_tokens = True, max_length=MAX_LEN*2, truncation=True)
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
      feats.append(self.get_feature(features))
      # feat.append(int(feature))

      idx_map[count] = [premise, hypothesis]
      idx.append(count)
      count+=1
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)

    y = torch.tensor(y)
    idx = torch.tensor(idx)
    feats = torch.tensor(feats)
    print(feats)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, feats, y, idx)
    return dataset, idx_map

  def get_data_loaders(self, batch_size=4, batches_per_epoch=402, shuffle=True): #1609 samples / 64:25=1600 / 402:4=1608
    train_loader_torch = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size,
    )

    val_loader_torch = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size,
    )

    test_loader_torch = DataLoader(
      self.test_data,
      shuffle=False,#shuffle,
      batch_size=batch_size,
    )
    
    train_loader = LoaderWrapper(train_loader_torch, n_step=batches_per_epoch)
    val_loader = LoaderWrapper(val_loader_torch, n_step=batches_per_epoch)
    test_loader = LoaderWrapper(test_loader_torch, n_step=batches_per_epoch)

    return train_loader, val_loader_torch, test_loader_torch

In [49]:
class LoaderWrapper:
    def __init__(self, loader, n_step):
        self.step = n_step
        self.idx = 0
        self.iter_loader = iter(loader)
        self.loader = loader
    
    def __iter__(self):
        return self

    def __len__(self):
        return self.step

    def __next__(self):
        # if reached number of steps desired, stop
        if self.idx == self.step:
            self.idx = 0
            raise StopIteration
        else:
            self.idx += 1
        # while True
        try:
            return next(self.iter_loader)
        except StopIteration:
            # reinstate iter_loader, then continue
            self.iter_loader = iter(self.loader)
            return next(self.iter_loader)

In [50]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df)

train_loader, val_loader, test_loader = mnli_dataset.get_data_loaders(batch_size=batch_size, batches_per_epoch=batches_per_epoch) #64X250
label_dict = mnli_dataset.label_dict # required by custom func to calculate accuracy, bert model
rev_label_dict = mnli_dataset.rev_label_dict # required by custom func to calculate accuracy

} Aesthetic Appreciation and Spanish Art : right
[198, 13492, 13542, 1105, 8492, 2944, 1893, 131, 1268]
Aesthetic Appreciation and Spanish Art :
[13492, 13542, 1105, 8492, 2944, 1893, 131]
} Claire Bailey-Ross claire.bailey-ross@port.ac.uk University of Portsmouth , United Kingdom right
[198, 172, 20293, 1162, 17076, 2254, 118, 187, 13159, 172, 20293, 1162, 119, 17076, 2254, 118, 187, 13159, 137, 4104, 119, 170, 1665, 119, 26006, 2755, 1104, 9267, 12462, 117, 10280, 6139, 1268]
Claire Bailey-Ross claire.bailey-ross@port.ac.uk University of Portsmouth , United Kingdom
[172, 20293, 1162, 17076, 2254, 118, 187, 13159, 172, 20293, 1162, 119, 17076, 2254, 118, 187, 13159, 137, 4104, 119, 170, 1665, 119, 26006, 2755, 1104, 9267, 12462, 117, 10280, 6139]
Claire Bailey-Ross claire.bailey-ross@port.ac.uk University of Portsmouth , United Kingdom
[172, 20293, 1162, 17076, 2254, 118, 187, 13159, 172, 20293, 1162, 119, 17076, 2254, 118, 187, 13159, 137, 4104, 119, 170, 1665, 119, 26006, 2755, 1104

In [ ]:
print(label_dict)

{'antithesis': 0, 'preparation': 1, 'justify': 2, 'contrast': 3, 'background': 4, 'evidence': 5, 'manner': 6, 'elaboration': 7, 'attribution': 8, 'purpose': 9, 'joint': 10, 'result': 11, 'solutionhood': 12, 'circumstance': 13, 'question': 14, 'motivation': 15, 'condition': 16, 'cause': 17, 'sequence': 18, 'restatement': 19, 'evaluation': 20, 'concession': 21, 'means': 22}


# Define the model

## load pretrained model

In [ ]:
from transformers import BertForSequenceClassification, AdamW
from torch import optim

# model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=len(label_dict)).to(device)
# optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.6, mode='max', patience=2, min_lr=5e-7, verbose=True)

In [ ]:
from CategoricalAccuracy import CategoricalAccuracy as CA
import numpy as np

ca = CA()

x = torch.tensor(np.array([[[1,0,0], [1,0,0], [1,0,0]]]))
y1 = torch.tensor(np.array([[0], [1], [1]]))
y2 = torch.tensor(np.array([[0], [0], [0]]))

ca(x,y1)
print(ca.get_metric(reset=True))
ca(x,y2)
print(ca.get_metric(reset=True))

0.3333333333333333
1.0


## define evaulation metric

In [ ]:
# to evaluate model for train and test. And also use classification report for testing
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

# helper function to calculate the batch accuracy
def multi_acc(y_pred, y_test, allennlp=False):
  if allennlp==False:
    acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
    return acc

def save_cm(y_true, y_pred, target_names, display_zeros=True):
  confusion_matrix = pd.crosstab(y_true, y_pred)
  print(confusion_matrix)
  target_namesx = confusion_matrix.columns
  target_namesy = confusion_matrix.index.values

  fig, ax = plt.subplots(figsize=(10,10))
  s = sns.heatmap(confusion_matrix, xticklabels=target_namesx, yticklabels=target_namesy, annot=True, fmt = '.5g')

  plt.title('CM predicted v actual values')
  plt.xlabel('Pred')
  plt.ylabel('True')
  plt.tight_layout()
  # plt.show()
  # plt.savefig(image_file+exp+'.png')
  # print(image_file+exp+'.png')

# freeze model weights and measure validation / test 
def evaluate_accuracy(model, optimizer, data_loader, rev_label_dict, label_dict, save_path, is_training=True):
  model.eval()
  total_val_acc  = 0
  total_val_loss = 0
  
  #for classification report
  y_true = []
  y_pred = []
  idx_list = []
  premise_list = []
  hypo_list = []
  idx_map = mnli_dataset.val_idx if is_training else mnli_dataset.test_idx

  with torch.no_grad():
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, feat, y, idx) in enumerate(data_loader):      
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      feat = feat.to(device)
      
      # loss, prediction = model(pair_token_ids, 
      #                       token_type_ids=seg_ids, 
      #                       attention_mask=mask_ids, 
      #                       labels=labels).values()
      # acc = multi_acc(prediction, labels)

      ############new code#####################

      outputs = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids, 
                            feat=feat)
      # probs = F.softmax(outputs, dim=1)
      # max_idx = torch.max(outputs, 1).indices
      # one_hot = F.one_hot(max_idx, outputs.shape[1])

      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, labels)
      acc = multi_acc(outputs, labels)
      ########################################

      total_val_loss += loss.item()
      total_val_acc  += acc.item()

      # log predictions for classification report
      argmax_predictions = torch.argmax(outputs,dim=1).tolist()
      labels_list = labels.tolist()
      assert(len(labels_list)==len(argmax_predictions))
      for p in argmax_predictions: y_pred.append(rev_label_dict[int(p)])
      for l in labels_list: y_true.append(rev_label_dict[l])
      for i in idx.tolist():
        idx_list.append(i)
        premise_list.append(idx_map[i][0])
        hypo_list.append(idx_map[i][1])

  val_acc  = total_val_acc/len(data_loader)
  val_loss = total_val_loss/len(data_loader)
  cr = classification_report(y_true, y_pred)

  idx_json = {'idx': idx_list, 'gold_label': y_true, 'pred_label': y_pred, 'premise': premise_list, 'hypothesis': hypo_list}
  # if not is_training: json.dump(idx_json, open(save_path, 'w', encoding='utf8'), ensure_ascii=False)

  # if not is_training:
  #   save_cm(y_true, y_pred, rev_label_dict)
  
  return val_acc, val_loss, cr, model, optimizer

## define bert custom model

In [ ]:
from transformers import BertModel, AutoTokenizer
import torch.nn as nn
class CustomBERTModel(nn.Module):
    #https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model
    def __init__(self, num_labels):
          super(CustomBERTModel, self).__init__()
          self.num_classes = num_labels+1 # zero indexed classes
          print('ASSIGN:', self.num_classes)
          self.bert = BertModel.from_pretrained(BERT_MODEL)
          ### New layers:
          self.linear1 = nn.Linear(768, 512)
        #   self.linear2 = nn.Linear(512, 256)
        #   self.linear3 = nn.Linear(256, 128)
        #   self.linear4 = nn.Linear(128, self.num_classes)
          self.act1 = nn.ReLU() # can i use the same activation object everywhere?
        #   self.act2 = nn.ReLU()
        #   self.act3 = nn.ReLU()
          self.drop = nn.Dropout(0.1) 

    def forward(self, pair_token_ids, token_type_ids, attention_mask, feat):
        sequence_output, pooled_output = self.bert(input_ids=pair_token_ids, 
                        token_type_ids=token_type_ids, 
                        attention_mask=attention_mask).values()

        feat = self.drop(feat)
        feat_concat = torch.concat((sequence_output[:,0,:].view(-1,768), feat),-1)

        # sequence_output has the following shape: (batch_size, sequence_length, 768)
        linear1_output = self.linear1(sequence_output[:,0,:].view(-1,768)) ## extract the 1st token's embeddings
        # linear1_output = self.linear1(sequence_output[:,0,:].view(-1,768)) ## extract the 1st token's embeddings
        linear1_output = self.act1(linear1_output)
        # linear2_output = self.linear2(linear1_output)
        # linear2_output = self.act2(linear2_output)
        # linear3_output = self.linear3(linear2_output)
        # linear3_output = self.act3(linear3_output)
        # linear4_output = self.linear4(linear3_output)
        # drop_output = self.drop(linear4_output)
        return linear1_output# loss, outputs

# tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
model = CustomBERTModel(mnli_dataset.num_labels) # You can pass the parameters if required to have more flexible model
model.to(device) ## can be gpu
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.6, mode='max', patience=2, min_lr=5e-7, verbose=True)#original factor=0.6, min_lr=5e-7

ASSIGN: 24


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated

## define training regime

In [ ]:
### MODIFIED
import time
import traceback
import torch.nn.functional as F
from typing import Optional, Iterable, Dict, Any
from EarlyStopperUtil import MetricTracker


from sklearn.metrics import classification_report

EPOCHS = 100

def train(model, train_loader, val_loader, optimizer, scheduler, rev_label_dict):  
  EarlyStopper = MetricTracker(patience=12, metric_name='+accuracy')

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0

    # logging for scheduler
    losses = []
    accuracies= []

    train_size = 0

    for batch_idx, (pair_token_ids, mask_ids, seg_ids, feat, y, idx) in enumerate(train_loader):
      train_size+=1
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      feat = feat.to(device)

      ############new code#####################

      outputs = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids,
                            feat=feat)
      # outputs = F.log_softmax(outputs, dim=1) # log prob
      # outputs = np.argmax(prob, axis=1) # preds
      # https://stackoverflow.com/questions/43672047/convert-probability-vector-into-target-vector-in-python
      # https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, labels)
      loss.backward()
      acc = multi_acc(outputs, labels)
      optimizer.step()
      ################old code#################

      # loss, prediction = model(pair_token_ids, 
      #                       token_type_ids=seg_ids, 
      #                       attention_mask=mask_ids, 
      #                       labels=labels).values()

      # acc = multi_acc(prediction, labels)
      # loss.backward()
      # optimizer.step()

      ########################################
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

      # log losses for scheduler
      losses.append(loss)
      accuracies.append(acc)
      
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)

    val_acc, val_loss, cr, model, optimizer = evaluate_accuracy(model, optimizer, val_loader, rev_label_dict, label_dict, None)
    EarlyStopper.add_metric(val_acc)
    if EarlyStopper.should_stop_early(): break

    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    print(f'train_size: {train_size}')


In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    train(model, train_loader, val_loader, optimizer, scheduler, rev_label_dict)

Epoch 1: train_loss: 2.7515 train_acc: 0.3891 | val_loss: 2.1816 val_acc: 0.4833
00:02:36.42
train_size: 1700
Epoch 2: train_loss: 1.8944 train_acc: 0.5493 | val_loss: 2.0334 val_acc: 0.5149
00:04:57.15
train_size: 1700


In [ ]:
save_path = 'bert-disrpt_nld_'+str(batch_size)+'X'+str(batches_per_epoch)+'.pt'
# torch.save(model.state_dict(), save_path)

# test

In [ ]:
def validate(model, test_loader, optimizer, rev_label_dict, label_dict, save_path):
  start = time.time()
  test_acc, test_loss, cr, model, optimizer = evaluate_accuracy(model, optimizer, test_loader, rev_label_dict, label_dict, save_path.replace('.pt', '.json'), is_training=False)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)

  print(f'Test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  print(cr)

  return test_loss, test_acc


# model.load_state_dict(torch.load(save_path))
test_loss, test_acc = validate(model, test_loader, optimizer, rev_label_dict, label_dict, save_path)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test_loss: 1.8774 test_acc: 0.5370
00:00:28.69
              precision    recall  f1-score   support

  antithesis       0.45      0.26      0.33        38
 attribution       0.74      0.85      0.79       101
  background       0.55      0.40      0.46        96
       cause       0.49      0.47      0.48        38
circumstance       0.67      0.70      0.68        74
  concession       0.27      0.49      0.35        51
   condition       0.00      0.00      0.00        47
    contrast       0.38      0.46      0.42        56
 elaboration       0.64      0.68      0.66       531
  evaluation       0.61      0.30      0.40       115
    evidence       0.60      0.41      0.48        76
       joint       0.43      0.81      0.56       346
     justify       0.20      0.02      0.04        49
      manner       0.00      0.00      0.00        21
       means       0.00      0.00      0.00        17
  motivation       0.00      0.00      0.00        14
 preparation       0.62      0.86 

/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,